In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pathlib
import os

# Set paths to your dataset
dataset_dir = r'C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\Dataset'

# Define image size and batch size
img_height, img_width = 224, 224  # Using 224x224 for ResNet101
batch_size = 32

# Create training and validation datasets
train_ds = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

validation_ds = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Get class names
class_names = train_ds.class_names
print(class_names)

# Define the model
resnet_model = models.Sequential()
pretrained_model = ResNet101(include_top=False, input_shape=(img_height, img_width, 3), pooling='avg', weights='imagenet')
for layer in pretrained_model.layers:
    layer.trainable = False

resnet_model.add(pretrained_model)
resnet_model.add(layers.Flatten())
resnet_model.add(layers.Dense(512, activation='relu'))
resnet_model.add(layers.Dense(256, activation='relu'))
resnet_model.add(layers.Dense(128, activation='relu'))
resnet_model.add(layers.Dense(len(class_names), activation='softmax'))

# Compile the model
resnet_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = resnet_model.fit(train_ds, validation_data=validation_ds, epochs=50)

# Evaluate the model
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()



In [ ]:
sample_image_path = None
for img_path in pathlib.Path(dataset_dir).rglob('20230928_132739.jpg'):
    sample_image_path = img_path
    break

if sample_image_path is not None:
    image = cv2.imread(str(sample_image_path))
    image_resized = cv2.resize(image, (img_height, img_width))
    image = np.expand_dims(image_resized, axis=0)

    # Make predictions
    image_pred = resnet_model.predict(image)

    # Produce a human-readable output label
    image_output_class = class_names[np.argmax(image_pred)]
    print("The predicted class is", image_output_class)
else:
    print("No images found in the validation directory.")

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

# Function to load and preprocess the image
def load_and_preprocess_image(image_path, img_height, img_width):
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image array
    return img_array

# Path to the new image
new_image_path = 'C:/Users/User/Desktop/back up/my doc/RESEARCH/CINNAMON CODES/Dataset/test.jpeg'

# Load and preprocess the image
img_height, img_width = 224, 224  # Use the same size as during training
new_image_array = load_and_preprocess_image(new_image_path, img_height, img_width)

# Make predictions
predictions = resnet_model.predict(new_image_array)
predicted_class = class_names[np.argmax(predictions)]
print(f"The predicted class is {predicted_class}")

# Display the image and prediction
plt.imshow(image.load_img(new_image_path))
plt.title(f"Predicted class: {predicted_class}")
plt.show()